In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Theta Space.ipynb")

HSP_T (generic function with 1 method)

In [2]:
using QuantumOptics

In [16]:
# Operator form of SP with Theta Phases 

function Op_Sp(Nx, Ny, alpha, Tx, Ty)
    
    N = Nx*Ny
    
    H_T = HSP_T(Nx, Ny, alpha, Tx,Ty)
    
    basis = NLevelBasis(N) #SP Basis
    
    H = SparseOperator(basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(basis, m, n)
        end
    end
    
    return H
end

Op_Sp (generic function with 1 method)

In [23]:
Nx=3;Ny=3;N=Nx*Ny;q=3;PN=2
Tx=Ty=2
op = Op_Sp(Nx, Ny, 1/q, Tx, Ty)

Operator(dim=9x9)
  basis: NLevel(N=9)sparse([2, 3, 4, 7, 1, 3, 5, 8, 1, 2  …  8, 9, 2, 5, 7, 9, 3, 6, 7, 8], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  7, 7, 8, 8, 8, 8, 9, 9, 9, 9], ComplexF64[-1.0 + 0.0im, 0.4161468365471424 - 0.9092974268256817im, -1.0 + 0.0im, 0.4161468365471424 + 0.9092974268256817im, -1.0 + 0.0im, -1.0 + 0.0im, 0.4999999999999998 - 0.8660254037844387im, 0.579401252953291 - 0.8150424455671965im, 0.4161468365471424 + 0.9092974268256817im, -1.0 + 0.0im  …  -1.0 + 0.0im, 0.4161468365471424 - 0.9092974268256817im, 0.579401252953291 + 0.8150424455671965im, 0.4999999999999998 + 0.8660254037844387im, -1.0 + 0.0im, -1.0 + 0.0im, -0.9955480895004333 + 0.09425498125848553im, 0.5000000000000004 - 0.8660254037844386im, 0.4161468365471424 + 0.9092974268256817im, -1.0 + 0.0im], 9, 9)

In [24]:
using LinearAlgebra

# check Op. form
eigenenergies(dense(op)) == eigvals(HSP_T(Nx, Ny, 1/q, Tx, Ty))

true

In [25]:
basis = NLevelBasis(N)
states = fermionstates(basis, PN)
basis_mb = ManyBodyBasis(basis, states)
H_mb = manybodyoperator(basis_mb, op)

Operator(dim=36x36)
  basis: ManyBody(onebodybasis=NLevel(N=9), states:36)sparse([2, 4, 7, 9, 10, 13, 1, 5, 8, 9  …  26, 31, 34, 36, 15, 20, 30, 32, 34, 35], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2  …  35, 35, 35, 35, 36, 36, 36, 36, 36, 36], ComplexF64[-1.0 + 0.0im, 0.4999999999999998 - 0.8660254037844387im, 0.579401252953291 - 0.8150424455671965im, 0.4161468365471424 - 0.9092974268256817im, -1.0 + 0.0im, 0.4161468365471424 + 0.9092974268256817im, -1.0 + 0.0im, 0.5000000000000004 + 0.8660254037844386im, -0.9955480895004333 - 0.09425498125848553im, -1.0 + 0.0im  …  -1.0 + 0.0im, 0.5000000000000004 - 0.8660254037844386im, -1.0 + 0.0im, -1.0 + 0.0im, 0.579401252953291 + 0.8150424455671965im, -0.9955480895004333 + 0.09425498125848553im, 0.4999999999999998 + 0.8660254037844387im, 0.5000000000000004 - 0.8660254037844386im, 0.4161468365471424 + 0.9092974268256817im, -1.0 + 0.0im], 36, 36)

In [26]:
# check MB (PN=1)
eigenenergies(dense(H_mb)) == eigvals(HSP_T(Nx, Ny, 1/q, Tx, Ty))

false

In [27]:
# Twisted Angle Space for Calculating Real Space Chern Numbers

Tsize=20
dx=2*pi/Tsize
dy=dx
Tx=collect(range(start=0, stop=2*pi, step=dx))
Ty=collect(range(start=0, stop=2*pi, step=dy))

21-element Vector{Float64}:
 0.0
 0.3141592653589793
 0.6283185307179586
 0.9424777960769379
 1.2566370614359172
 1.5707963267948966
 1.8849555921538759
 2.199114857512855
 2.5132741228718345
 2.827433388230814
 3.141592653589793
 3.4557519189487724
 3.7699111843077517
 4.084070449666731
 4.39822971502571
 4.71238898038469
 5.026548245743669
 5.340707511102648
 5.654866776461628
 5.969026041820607
 6.283185307179586

In [28]:
function Chern_Nums(Tx, Ty, n1, n2)
    
    Sum=0
    
    for tx in range(start=1, stop=length(Tx))
        for ty in range(start=1, stop=length(Ty))
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty])
            H_mb = manybodyoperator(basis_mb, op)
            w1, v1 = eigen(dense(H_mb).data)
            v1 = v1[:,n1:n2]
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty])
            H_mb = manybodyoperator(basis_mb, op)
            w2, v2 = eigen(dense(H_mb).data)
            v2 = v2[:,n1:n2]
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx], Ty[ty]+dy)
            H_mb = manybodyoperator(basis_mb, op)
            w3, v3 = eigen(dense(H_mb).data)
            v3 = v3[:,n1:n2]
            #------------------------------------
            op = Op_Sp(Nx, Ny, 1/q, Tx[tx]+dx, Ty[ty]+dy)
            H_mb = manybodyoperator(basis_mb, op)
            w4, v4 = eigen(dense(H_mb).data)
            v4 = v4[:,n1:n2]
            #----------LINK VARIABLES------------
            U1=det(adjoint(v1)*v2)
            U1=U1/abs(U1)
            U2=det(adjoint(v2)*v4)
            U2=U2/abs(U2)
            U3=det(adjoint(v3)*v4)
            U3=U3/abs(U3)
            U4=det(adjoint(v1)*v3)
            U4=U4/abs(U4)
            #----------BERRY CURVATURE-----------
            F=log(U1*U2*1/U3*1/U4)
            Sum=Sum+F
        end
    end
    
    return 1/(2*pi*1im)*Sum
end

#Nx=Ny=3 iken:
#n1=1;n2=12
#n3=13;n4=24
#n5=25;n6=36

Chern_Nums(Tx, Ty, 1, 12)

2.730402245122732 + 1.1790298986318535e-16im